In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

In [ ]:
import os

pwd = os.getcwd()
pwd

In [ ]:
from VIDataloader.VIDataloader import get_dataloader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((540, 960)), transforms.ToTensor(),
])
image_dir = os.path.join(pwd, "train_images", "images")
mask_dir = os.path.join(pwd, "train_images", "masks")
batch_size = 4

dataloader = get_dataloader(image_dir, mask_dir, batch_size, transform)
type(dataloader), len(dataloader)

In [ ]:
class Object(object):
    pass
opt = Object()
opt.t_stride = 5

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from model import generate_model
from train import train

criterion_L1 = nn.L1Loss()

def criterion_ssim(output, target):
    ssim_loss = 1 - F.ssim(output, target, data_range=1, win_size=11, size_average=True)
    return ssim_loss

def get_optimizer(model, lr=0.001):
    return optim.Adam(model.parameters(), lr=lr)

model = generate_model(opt)
train(10, dataloader, model, criterion_L1, criterion_ssim, get_optimizer, opt)

In [ ]:
torch.save(model.state_dict(), 'model_weights.pt')